# Summary Forecast and Receipts

This notebook provides a summary roll-up of retail dollars by Category for 2025 and 2026.

**Metrics included:**
- Demand Forecast
- Expected Sales
- Forecasted Sales
- Missed Demand
- Proposed Receipts

## Setup and Imports

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Load Data Files

In [2]:
import os

# Get the directory where this notebook is located
base_path = os.path.dirname(os.path.abspath('__file__'))
data_path = os.path.join(base_path, '..', 'data')

# Load demand forecast file
demand_forecast = pd.read_csv(os.path.join(data_path, 'demand_forecast_file.csv'))
print(f"Demand forecast loaded: {len(demand_forecast):,} rows")

# Load sales forecast file (contains expected sales, forecast sales, missed demand)
sales_forecast = pd.read_csv(os.path.join(data_path, 'sales_forecast_output.csv'))
print(f"Sales forecast loaded: {len(sales_forecast):,} rows")

# Load proposed receipts file
proposed_receipts = pd.read_csv(os.path.join(data_path, 'proposed_receipts_output.csv'))
print(f"Proposed receipts loaded: {len(proposed_receipts):,} rows")

# Load catalog data
catalog = pd.read_csv(os.path.join(data_path, 'cz_catalog_data.csv'))
print(f"Catalog loaded: {len(catalog):,} rows")

Demand forecast loaded: 293,232 rows
Sales forecast loaded: 54,376 rows
Proposed receipts loaded: 1,139 rows
Catalog loaded: 6,109 rows


## Data Preparation

In [3]:
# Clean and prepare demand forecast data
demand_forecast['MONTH'] = pd.to_datetime(demand_forecast['MONTH'])
demand_forecast['SKU'] = demand_forecast['SKU'].astype(str)
demand_forecast['UNIT DEMAND'] = pd.to_numeric(demand_forecast['UNIT DEMAND'], errors='coerce').fillna(0)

# Clean sales forecast data
sales_forecast['MONTH'] = pd.to_datetime(sales_forecast['MONTH'])
sales_forecast['SKU'] = sales_forecast['SKU'].astype(str)
sales_forecast['EXPECTED UNIT SALES'] = pd.to_numeric(sales_forecast['EXPECTED UNIT SALES'], errors='coerce').fillna(0)
sales_forecast['FORECAST UNIT SALES'] = pd.to_numeric(sales_forecast['FORECAST UNIT SALES'], errors='coerce').fillna(0)
sales_forecast['MISSED DEMAND'] = pd.to_numeric(sales_forecast['MISSED DEMAND'], errors='coerce').fillna(0)

# Clean proposed receipts data
proposed_receipts['SKU'] = proposed_receipts['SKU'].astype(str)
proposed_receipts['PROPOSED_RECEIPTS'] = pd.to_numeric(proposed_receipts['PROPOSED_RECEIPTS'], errors='coerce').fillna(0)

# Clean catalog data
catalog['SKU'] = catalog['SKU'].astype(str)
catalog['FULL_PRICE_RETAIL'] = pd.to_numeric(catalog['FULL_PRICE_RETAIL'], errors='coerce').fillna(0)
catalog['CATEGORY'] = catalog['CATEGORY'].fillna('UNKNOWN').astype(str)

# Filter to 2025 and 2026 only
demand_forecast['YEAR'] = demand_forecast['MONTH'].dt.year
demand_forecast_filtered = demand_forecast[demand_forecast['YEAR'].isin([2025, 2026])].copy()

sales_forecast['YEAR'] = sales_forecast['MONTH'].dt.year
sales_forecast_filtered = sales_forecast[sales_forecast['YEAR'].isin([2025, 2026])].copy()

print(f"Demand forecast filtered to 2025-2026: {len(demand_forecast_filtered):,} rows")
print(f"Sales forecast filtered to 2025-2026: {len(sales_forecast_filtered):,} rows")

# Create month name column for display
demand_forecast_filtered['MONTH_NAME'] = demand_forecast_filtered['MONTH'].dt.strftime('%b')
sales_forecast_filtered['MONTH_NAME'] = sales_forecast_filtered['MONTH'].dt.strftime('%b')

Demand forecast filtered to 2025-2026: 146,616 rows
Sales forecast filtered to 2025-2026: 54,376 rows


In [4]:
# Merge demand forecast with catalog to get CATEGORY and FULL_PRICE_RETAIL
demand_merged = demand_forecast_filtered.merge(
    catalog[['SKU', 'CATEGORY', 'FULL_PRICE_RETAIL']],
    on='SKU',
    how='left'
)
demand_merged['CATEGORY'] = demand_merged['CATEGORY'].fillna('UNKNOWN')
demand_merged['FULL_PRICE_RETAIL'] = demand_merged['FULL_PRICE_RETAIL'].fillna(0)
demand_merged['RETAIL_DEMAND_DOLLARS'] = demand_merged['UNIT DEMAND'] * demand_merged['FULL_PRICE_RETAIL']

# Merge sales forecast with catalog
sales_merged = sales_forecast_filtered.merge(
    catalog[['SKU', 'CATEGORY', 'FULL_PRICE_RETAIL']],
    on='SKU',
    how='left'
)
sales_merged['CATEGORY'] = sales_merged['CATEGORY'].fillna('UNKNOWN')
sales_merged['FULL_PRICE_RETAIL'] = sales_merged['FULL_PRICE_RETAIL'].fillna(0)
sales_merged['EXPECTED_SALES_DOLLARS'] = sales_merged['EXPECTED UNIT SALES'] * sales_merged['FULL_PRICE_RETAIL']
sales_merged['FORECAST_SALES_DOLLARS'] = sales_merged['FORECAST UNIT SALES'] * sales_merged['FULL_PRICE_RETAIL']
sales_merged['MISSED_DEMAND_DOLLARS'] = sales_merged['MISSED DEMAND'] * sales_merged['FULL_PRICE_RETAIL']

# Merge proposed receipts with catalog (include both FULL_PRICE_RETAIL and VENDOR_COST_USD)
receipts_merged = proposed_receipts.merge(
    catalog[['SKU', 'CATEGORY', 'FULL_PRICE_RETAIL', 'VENDOR_COST_USD']],
    on='SKU',
    how='left'
)
receipts_merged['CATEGORY'] = receipts_merged['CATEGORY'].fillna('UNKNOWN')
receipts_merged['FULL_PRICE_RETAIL'] = pd.to_numeric(receipts_merged['FULL_PRICE_RETAIL'], errors='coerce').fillna(0)
receipts_merged['VENDOR_COST_USD'] = pd.to_numeric(receipts_merged['VENDOR_COST_USD'], errors='coerce').fillna(0)
receipts_merged['PROPOSED_RECEIPTS_RETAIL'] = receipts_merged['PROPOSED_RECEIPTS'] * receipts_merged['FULL_PRICE_RETAIL']
receipts_merged['PROPOSED_RECEIPTS_PO_COST'] = receipts_merged['PROPOSED_RECEIPTS'] * receipts_merged['VENDOR_COST_USD']

# Put all proposed receipts under current month (December 2025)
from datetime import datetime
current_date = datetime.now()
receipts_merged['YEAR'] = current_date.year
receipts_merged['MONTH_NAME'] = current_date.strftime('%b')

print(f"Demand merged: {len(demand_merged):,} rows")
print(f"Sales merged: {len(sales_merged):,} rows")
print(f"Receipts merged: {len(receipts_merged):,} rows")
print(f"\nProposed Receipts Total Retail: ${receipts_merged['PROPOSED_RECEIPTS_RETAIL'].sum():,.0f}")
print(f"Proposed Receipts Total PO Cost: ${receipts_merged['PROPOSED_RECEIPTS_PO_COST'].sum():,.0f}")
print(f"\nAll proposed receipts assigned to: {current_date.strftime('%b %Y')}")

Demand merged: 146,616 rows
Sales merged: 54,376 rows
Receipts merged: 1,139 rows

Proposed Receipts Total Retail: $13,695,288
Proposed Receipts Total PO Cost: $3,764,246

All proposed receipts assigned to: Dec 2025


## Create Pivot Tables by Category

In [5]:
def create_pivot_with_totals(data, index_col, values_col, month_col='MONTH_NAME', year_col='YEAR'):
    """
    Create a pivot table with monthly columns, year subtotals, and grand total.
    """
    # Create pivot table
    pivot = data.pivot_table(
        index=index_col,
        columns=[year_col, month_col],
        values=values_col,
        aggfunc='sum',
        fill_value=0
    )
    
    # Reorder months
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Create ordered columns with year totals AFTER each year's months
    ordered_cols = []
    for year in [2025, 2026]:
        # Add all months for this year
        for month in month_order:
            if (year, month) in pivot.columns:
                ordered_cols.append((year, month))
        # Calculate and add year total AFTER the months
        year_cols = [col for col in pivot.columns if col[0] == year]
        if year_cols:
            pivot[(year, 'TOTAL')] = pivot[year_cols].sum(axis=1)
            ordered_cols.append((year, 'TOTAL'))
    
    # Calculate grand total
    grand_total_cols = [(y, m) for (y, m) in ordered_cols if m != 'TOTAL']
    pivot[('GRAND', 'TOTAL')] = pivot[grand_total_cols].sum(axis=1)
    ordered_cols.append(('GRAND', 'TOTAL'))
    
    # Reorder pivot columns
    pivot = pivot[ordered_cols]
    
    # Add row totals
    totals_row = pivot.sum(axis=0)
    totals_row.name = 'TOTAL'
    pivot = pd.concat([pivot, totals_row.to_frame().T])
    
    return pivot

# Create pivot tables for each metric
demand_pivot = create_pivot_with_totals(demand_merged, 'CATEGORY', 'RETAIL_DEMAND_DOLLARS')
expected_sales_pivot = create_pivot_with_totals(sales_merged, 'CATEGORY', 'EXPECTED_SALES_DOLLARS')
forecast_sales_pivot = create_pivot_with_totals(sales_merged, 'CATEGORY', 'FORECAST_SALES_DOLLARS')
missed_demand_pivot = create_pivot_with_totals(sales_merged, 'CATEGORY', 'MISSED_DEMAND_DOLLARS')

# Create pivot tables for proposed receipts (now with monthly breakdown under current month)
receipts_retail_pivot = create_pivot_with_totals(receipts_merged, 'CATEGORY', 'PROPOSED_RECEIPTS_RETAIL')
receipts_po_cost_pivot = create_pivot_with_totals(receipts_merged, 'CATEGORY', 'PROPOSED_RECEIPTS_PO_COST')

print("Pivot tables created for all metrics")
print(f"Categories in demand pivot: {len(demand_pivot) - 1}")  # -1 for total row
print(f"\nColumn order: {list(demand_pivot.columns)}")

Pivot tables created for all metrics
Categories in demand pivot: 10

Column order: [(2025, 'Jan'), (2025, 'Feb'), (2025, 'Mar'), (2025, 'Apr'), (2025, 'May'), (2025, 'Jun'), (2025, 'Jul'), (2025, 'Aug'), (2025, 'Sep'), (2025, 'Oct'), (2025, 'Nov'), (2025, 'Dec'), (2025, 'TOTAL'), (2026, 'Jan'), (2026, 'Feb'), (2026, 'Mar'), (2026, 'Apr'), (2026, 'May'), (2026, 'Jun'), (2026, 'Jul'), (2026, 'Aug'), (2026, 'Sep'), (2026, 'Oct'), (2026, 'Nov'), (2026, 'Dec'), (2026, 'TOTAL'), ('GRAND', 'TOTAL')]


## Combine All Metrics

In [6]:
def combine_metrics_to_excel_df(demand_pivot, expected_pivot, forecast_pivot, missed_pivot, 
                                  receipts_retail_pivot, receipts_po_cost_pivot):
    """
    Combine all metrics into a single dataframe with Category and Metric columns.
    Each category will have multiple rows (one per metric).
    """
    all_rows = []
    
    # Get all categories (excluding TOTAL row)
    categories = [cat for cat in demand_pivot.index if cat != 'TOTAL']
    
    # Define metrics and their source pivots (now including proposed receipts as pivots)
    metrics = [
        ('Demand Forecast', demand_pivot),
        ('Expected Sales', expected_pivot),
        ('Forecasted Sales', forecast_pivot),
        ('Missed Demand', missed_pivot),
        ('Proposed Receipts (Retail)', receipts_retail_pivot),
        ('Proposed Receipts (PO Cost)', receipts_po_cost_pivot),
    ]
    
    # Get all columns from demand_pivot as the master column list
    master_columns = list(demand_pivot.columns)
    
    # Process each category
    for category in categories:
        for metric_name, pivot in metrics:
            row = {'CATEGORY': category, 'METRIC': metric_name}
            for col in master_columns:
                year, month = col
                if month == 'TOTAL' and year != 'GRAND':
                    col_name = f'{year} Total'
                elif year == 'GRAND':
                    col_name = 'Grand Total'
                else:
                    col_name = f'{year} {month}'
                
                # Check if category and column exist in this pivot
                if category in pivot.index and col in pivot.columns:
                    val = pivot.loc[category, col]
                else:
                    val = 0
                row[col_name] = 0 if pd.isna(val) else val
            all_rows.append(row)
    
    # Add TOTAL rows for each metric
    for metric_name, pivot in metrics:
        row = {'CATEGORY': 'TOTAL', 'METRIC': metric_name}
        for col in master_columns:
            year, month = col
            if month == 'TOTAL' and year != 'GRAND':
                col_name = f'{year} Total'
            elif year == 'GRAND':
                col_name = 'Grand Total'
            else:
                col_name = f'{year} {month}'
            
            if 'TOTAL' in pivot.index and col in pivot.columns:
                val = pivot.loc['TOTAL', col]
            else:
                val = 0
            row[col_name] = 0 if pd.isna(val) else val
        all_rows.append(row)
    
    # Create dataframe and replace any remaining NaN with 0
    df = pd.DataFrame(all_rows)
    df = df.fillna(0)
    
    return df

# Combine all metrics
excel_df = combine_metrics_to_excel_df(
    demand_pivot, 
    expected_sales_pivot, 
    forecast_sales_pivot, 
    missed_demand_pivot, 
    receipts_retail_pivot,
    receipts_po_cost_pivot
)

print(f"Combined dataframe: {len(excel_df)} rows, {len(excel_df.columns)} columns")
excel_df.head(18)

Combined dataframe: 66 rows, 29 columns


,CATEGORY,METRIC,2025 Jan,2025 Feb,2025 Mar,2025 Apr,2025 May,2025 Jun,2025 Jul,2025 Aug,2025 Sep,2025 Oct,2025 Nov,2025 Dec,2025 Total,2026 Jan,2026 Feb,2026 Mar,2026 Apr,2026 May,2026 Jun,2026 Jul,2026 Aug,2026 Sep,2026 Oct,2026 Nov,2026 Dec,2026 Total,Grand Total
0,ACCENTS,Demand Forecast,306827.292995,341438.558036,258488.693915,282068.249597,408972.250402,251739.555478,227191.823567,357784.858623,387005.064124,294819.464315,685077.692239,4.139678e+05,4.215381e+06,306827.292995,341438.558036,258488.693915,282068.249597,408972.250402,251739.555478,227191.823567,357784.858623,387005.064124,294819.464315,685077.692239,413967.846709,4215381.35,8.430763e+06
1,ACCENTS,Expected Sales,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90456.000000,5.120000e+04,1.416560e+05,185660.000000,156209.000000,77908.000000,54072.000000,49311.000000,25151.000000,16053.000000,8109.000000,8143.000000,10048.000000,18335.000000,7889.000000,616888.00,7.585440e+05
2,ACCENTS,Forecasted Sales,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,126292.000000,6.156100e+04,1.878530e+05,210067.000000,190897.000000,180835.000000,268582.000000,391706.000000,242573.000000,254663.000000,369646.000000,356322.000000,318299.000000,595401.000000,344581.000000,3723572.00,3.911425e+06
3,ACCENTS,Missed Demand,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,384418.000000,9.060500e+04,4.750230e+05,122896.000000,171731.000000,151403.000000,203440.000000,314578.000000,200056.000000,220747.000000,335150.000000,322367.000000,285379.000000,533487.000000,312212.000000,3173446.00,3.648469e+06
4,ACCENTS,Proposed Receipts (Retail),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.328510e+06,1.328510e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.328510e+06
5,ACCENTS,Proposed Receipts (PO Cost),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.890164e+05,3.890164e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,3.890164e+05
6,BASKETS,Demand Forecast,158446.272698,158722.818393,152258.196714,139086.246564,216598.548427,134333.298437,142921.326688,197754.623404,206812.737313,154172.969671,288961.780407,2.044532e+05,2.154522e+06,158446.272698,158722.818393,152258.196714,139086.246564,216598.548427,134333.298437,142921.326688,197754.623404,206812.737313,154172.969671,288961.780407,204453.181286,2154522.00,4.309044e+06
7,BASKETS,Expected Sales,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53065.000000,2.202500e+04,7.509000e+04,39159.000000,31920.000000,24038.000000,15995.000000,27968.000000,12692.000000,9228.000000,3052.000000,2823.000000,3121.000000,4631.000000,3240.000000,177867.00,2.529570e+05
8,BASKETS,Forecasted Sales,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54229.000000,2.224500e+04,7.647400e+04,40323.000000,41608.000000,136409.000000,126786.000000,193437.000000,121032.000000,148371.000000,191403.000000,176831.000000,153393.000000,230460.000000,158216.000000,1718269.00,1.794743e+06
9,BASKETS,Missed Demand,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,137946.000000,5.936400e+04,1.973100e+05,122820.000000,121250.000000,111427.000000,109847.000000,164305.000000,107396.000000,138199.000000,187187.000000,172844.000000,149108.000000,224445.000000,153812.000000,1762640.00,1.959950e+06


## Export to Excel

In [7]:
# Export to Excel with formatting
output_excel_file = os.path.join(data_path, 'Summary_Forecast_and_Receipts.xlsx')

with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    workbook = writer.book
    
    # Define formats
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#4472C4',
        'font_color': 'white',
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'text_wrap': True
    })
    
    year_total_format = workbook.add_format({
        'bold': True,
        'bg_color': '#E2EFDA',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    grand_total_format = workbook.add_format({
        'bold': True,
        'bg_color': '#FFC000',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    currency_format = workbook.add_format({
        'num_format': '$#,##0',
        'border': 1,
        'align': 'right'
    })
    
    text_format = workbook.add_format({
        'border': 1,
        'align': 'left'
    })
    
    total_row_format = workbook.add_format({
        'bold': True,
        'bg_color': '#FFC000',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    total_row_text_format = workbook.add_format({
        'bold': True,
        'bg_color': '#FFC000',
        'border': 1,
        'align': 'left'
    })
    
    # Category separator format (light gray background for first row of each category)
    category_first_format = workbook.add_format({
        'bg_color': '#D9E1F2',
        'border': 1,
        'align': 'left'
    })
    
    category_first_currency = workbook.add_format({
        'bg_color': '#D9E1F2',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    category_first_year_total = workbook.add_format({
        'bold': True,
        'bg_color': '#C6EFCE',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    # Write data
    excel_df.to_excel(writer, sheet_name='By Category', index=False, startrow=0)
    worksheet = writer.sheets['By Category']
    
    # Set column widths to fit text
    worksheet.set_column('A:A', 20)  # Category column - wider
    worksheet.set_column('B:B', 26)  # Metric column - wider for "Proposed Receipts (Retail)"
    
    # Set widths for month columns and total columns
    for col_num, col_name in enumerate(excel_df.columns):
        if col_num <= 1:
            continue  # Skip Category and Metric columns (already set)
        elif 'Total' in str(col_name):
            worksheet.set_column(col_num, col_num, 16)  # Wider for total columns
        else:
            worksheet.set_column(col_num, col_num, 14)  # Month columns
    
    # Apply header formatting
    for col_num, value in enumerate(excel_df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    # Apply data formatting
    for row_num in range(1, len(excel_df) + 1):
        is_total_row = excel_df.iloc[row_num - 1]['CATEGORY'] == 'TOTAL'
        is_first_metric = excel_df.iloc[row_num - 1]['METRIC'] == 'Demand Forecast'
        
        for col_num, col_name in enumerate(excel_df.columns):
            value = excel_df.iloc[row_num - 1, col_num]
            
            if is_total_row:
                if col_name in ['CATEGORY', 'METRIC']:
                    worksheet.write(row_num, col_num, value, total_row_text_format)
                elif 'Grand' in str(col_name):
                    worksheet.write(row_num, col_num, value, grand_total_format)
                else:
                    worksheet.write(row_num, col_num, value, total_row_format)
            elif is_first_metric:
                if col_name in ['CATEGORY', 'METRIC']:
                    worksheet.write(row_num, col_num, value, category_first_format)
                elif 'Total' in str(col_name) and 'Grand' not in str(col_name):
                    worksheet.write(row_num, col_num, value, category_first_year_total)
                elif 'Grand' in str(col_name):
                    worksheet.write(row_num, col_num, value, grand_total_format)
                else:
                    worksheet.write(row_num, col_num, value, category_first_currency)
            else:
                if col_name in ['CATEGORY', 'METRIC']:
                    worksheet.write(row_num, col_num, value, text_format)
                elif 'Total' in str(col_name) and 'Grand' not in str(col_name):
                    worksheet.write(row_num, col_num, value, year_total_format)
                elif 'Grand' in str(col_name):
                    worksheet.write(row_num, col_num, value, grand_total_format)
                else:
                    worksheet.write(row_num, col_num, value, currency_format)
    
    # Freeze headers and first two columns
    worksheet.freeze_panes(1, 2)

print(f"\n✓ Excel file exported to: {output_excel_file}")
print(f"  Sheet: 'By Category'")
print(f"  Metrics: Demand Forecast, Expected Sales, Forecasted Sales, Missed Demand, Proposed Receipts (Retail), Proposed Receipts (PO Cost)")


✓ Excel file exported to: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\Summary_Forecast_and_Receipts.xlsx
  Sheet: 'By Category'
  Metrics: Demand Forecast, Expected Sales, Forecasted Sales, Missed Demand, Proposed Receipts (Retail), Proposed Receipts (PO Cost)


## Summary Statistics

In [8]:
# Display summary statistics
print("=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

# Total demand by year
yearly_demand = demand_merged.groupby('YEAR')['RETAIL_DEMAND_DOLLARS'].sum()
print(f"\nTotal Retail Demand Dollars:")
for year, total in yearly_demand.items():
    print(f"  {year}: ${total:,.0f}")
print(f"  Grand Total: ${yearly_demand.sum():,.0f}")

# Total expected sales by year
yearly_expected = sales_merged.groupby('YEAR')['EXPECTED_SALES_DOLLARS'].sum()
print(f"\nTotal Expected Sales Dollars:")
for year, total in yearly_expected.items():
    print(f"  {year}: ${total:,.0f}")
print(f"  Grand Total: ${yearly_expected.sum():,.0f}")

# Total forecast sales by year
yearly_forecast = sales_merged.groupby('YEAR')['FORECAST_SALES_DOLLARS'].sum()
print(f"\nTotal Forecasted Sales Dollars:")
for year, total in yearly_forecast.items():
    print(f"  {year}: ${total:,.0f}")
print(f"  Grand Total: ${yearly_forecast.sum():,.0f}")

# Total missed demand by year
yearly_missed = sales_merged.groupby('YEAR')['MISSED_DEMAND_DOLLARS'].sum()
print(f"\nTotal Missed Demand Dollars:")
for year, total in yearly_missed.items():
    print(f"  {year}: ${total:,.0f}")
print(f"  Grand Total: ${yearly_missed.sum():,.0f}")

# Total proposed receipts
print(f"\nTotal Proposed Receipts (Retail): ${receipts_merged['PROPOSED_RECEIPTS_RETAIL'].sum():,.0f}")
print(f"Total Proposed Receipts (PO Cost): ${receipts_merged['PROPOSED_RECEIPTS_PO_COST'].sum():,.0f}")

SUMMARY STATISTICS

Total Retail Demand Dollars:
  2025: $65,550,883
  2026: $65,550,883
  Grand Total: $131,101,766

Total Expected Sales Dollars:
  2025: $2,079,736
  2026: $5,169,738
  Grand Total: $7,249,474

Total Forecasted Sales Dollars:
  2025: $3,584,857
  2026: $50,588,089
  Grand Total: $54,172,946

Total Missed Demand Dollars:
  2025: $4,476,202
  2026: $36,066,558
  Grand Total: $40,542,760

Total Proposed Receipts (Retail): $13,695,288
Total Proposed Receipts (PO Cost): $3,764,246
